!["Anaconda"](img/anaconda-logo.png)

*Copyright Continuum 2012-2016 All Rights Reserved.*

# Dask: Introduction


<img src="img/dask_horizontal.svg" align="right" width="30%">

Dask is a library for parallel computing on larger-than-memory data.

Dask emphasizes the following virtues:
*  **Familiar**: Provides parallelized NumPy array and Pandas DataFrame objects
*  **Native**: Enables distributed computing in Pure Python with access to the PyData stack.
*  **Fast**: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
*  **Flexible**: Supports complex and messy workloads
*  **Scales up**: Runs resiliently on clusters with 100s of nodes
*  **Scales down**: Trivial to set up and run on a laptop in a single process
*  **Responsive**: Designed with interactive computing in mind it provides rapid feedback and diagnostics to aid humans

## Table of Contents
* [Dask: Introduction](#Dask:-Introduction)
* [Background](#Background)
	* [Data Structures](#Data-Structures)
	* [The Problem](#The-Problem)
	* [The Solution](#The-Solution)
* [Two Levels of Dask](#Two-Levels-of-Dask)
	* [High-Level](#High-Level)
	* [Low Level](#Low-Level)
* [High-Level Interfaces, Mimicry of Familiar Containers](#High-Level-Interfaces,-Mimicry-of-Familiar-Containers)
	* [Dask DataFrame](#Dask-DataFrame)
	* [Dask Array](#Dask-Array)
	* [Dask Bag](#Dask-Bag)
	* [Dask Delayed](#Dask-Delayed)
* [Low-Level Machinery: Task Graphs and Schedulers](#Low-Level-Machinery:-Task-Graphs-and-Schedulers)
	* [Graphs](#Graphs)
	* [Scheduling](#Scheduling)
	* [Profiling](#Profiling)
* [Demonstration](#Demonstration)
* [Further Reading](#Further-Reading)
	* [Reference Links](#Reference-Links)
	* [Dask Tutorial](#Dask-Tutorial)


# Background

## Data Structures

The PyData is largely a software community based on 2 data structures:
* Numpy Array (N-dimensional array)
* Pandas DataFrame (in-memory relational database table)

> *"The fact that we all depend on these 2 data structures makes it so that all of our libraries interoperate very effectively. 
We have plotting libraries, data loading libraries, data analysis libraries, 
All these libraries can inter-operate without having to communicate with each other. 
Developers on indepedent teams can work independently without worrying about what other dev groups are doing."* -- Matt Rocklin, core developer on the Dask project, at [PyData Seattle 2015 (YouTube)](https://www.youtube.com/watch?v=ieW3G7ZzRZ0)

## The Problem

These two data structures have two big flaws:
* Mostly only handle data that fits into memory (RAM)
* Mostly only use a single core

Problems:
* As hard-drives get big and support larger data files...
* we want to reach outside of memory and **use the hard-drive**, not just the RAM
* New machine have more and **more cores** we are not using
* Imbalance between what our core data structures can do, and how good hardware is getting

## The Solution

Dask is a library for parallel computing on larger-than-memory data
* local dask can handle data of size (1 GB < data < 1 TB)
    * using your laptop more effectively
* distributed dask can handle much more
    * running jobs on a cluster

# Two Levels of Dask

Two ways to interact with Dask
* high-level API (that mimics Numpy and Pandas)
* low-level machinery (Delayed computation of task graphs with dynamic, low-latency task scheduler)

Let's talk about both high and low level, interleaved.
* Talk about the high level pandas like thing, and then
* discussion about how that is enabled by the low level machinery.

From High to Low:
* Dask Array/DataFrame takes your high-level code, and turns it into a "recipe"
* This "recipe" is a task graph of a bunch of different small functions to run in memeory
* Dask schedulers then execute these task graphs in parallel, using many cores, many threads, many processes on many nodes in a cluster.

## High-Level

Dask at a High-Level interactions:
* Numpy mimic, called "Dask Array" (numpy array + threading)
* Pandas mimic, called "Dask DataFrame" (pandas dataframe + threading)
* Python (parallel) List mimic, called "Dask Bag" (map, filter, + multi-processing)
* All of these rely on the same low-level

We want to write code up at this high-level:

```python
import dask.dataframe as dd
df = dd.read_csv('2015-*.csv')
df.groupby(df.user_id).value.mean().compute()
```

## Low Level

Dask schedulers execute the task graphs in parallel, using many cores.


<img src="img/fail-case.gif" width=60% align="right">
Dask at a Low-Level technology:
* Dynamic and low-latency task scheduler
* Aware of memory
* Accessible
* Arbitrary and simple graph structure
* Dask schedulers provide custom parallelism


# High-Level Interfaces, Mimicry of Familiar Containers

* Dask collections are the main interaction point for users. 
* They look like NumPy and pandas but generate dask graphs internally. 
* If you are a dask *user* then you should start here.

## Dask DataFrame

The Dask DataFrame mimics the Pandas DataFrame API:

```python
# Pandas
import pandas as pd
df = pd.read_csv('2015-01-01.csv')
df.groupby(df.user_id).value.mean()
```

```python
# Dask Mimics Pandas
import dask.dataframe as dd
df = dd.read_csv('2015-*-*.csv')
df.groupby(df.user_id).value.mean().compute()
```

## Dask Array

The Dask Array mimics the Numpy ndarray API:

```python
# Numpy
import numpy as np 
f = h5py.File('myfile.hdf5')
x = np.array(f['/small-data'])
x - x.mean(axis=1)
```

```python
# Dask mimics Numpy
import dask.array as da
f = h5py.File('myfile.hdf5')
x = da.from_array(f['/big-data'], chunks=(1000, 1000))
x - x.mean(axis=1).compute()
```

## Dask Bag

The Dask Bag is like a Python parallel list, and mimics iterators, Toolz, PySpark

```python
import dask.bag as db
b = db.read_text('2015-*-*.json.gz').map(json.loads)
b.pluck('name').frequencies().topk(10, lambda pair: pair[1]).compute()
```

## Dask Delayed

The Dask Delayed structure mimics ***`for loops`*** and wraps custom code.

```python

from dask import delayed
L = []
for fn in filenames:            # Use for loops to build up computation
   data = delayed(load)(fn)          # Delay execution of function
   L.append(delayed(process)(data))  # Build connections between variables

result = delayed(summarize)(L)
result.compute()
```

# Low-Level Machinery: Task Graphs and Schedulers

Dask represents parallel computations with [task graphs](graphs.md).  

These directed acyclic graphs (DAG) may have arbitrary structure

* freedom to build sophisticated algorithms
* can handle messy situations not easily managed by the ``map/filter/groupby`` paradigm common in most data engineering frameworks.


![Dask collections and schedulers](img/collections-schedulers.png)

## Graphs

Dask Graphs

* Dask graphs encode algorithms in a simple format involving Python dicts, tuples, and functions.
* This graph format can be used in isolation from the dask collections.
* Working directly with dask graphs is an excellent way to implement and test new algorithms in fields such as linear algebra, optimization, and machine learning.  


If you are a *developer*, you should start here.


## Scheduling

Schedulers execute task graphs.  

Dask currently has two main schedulers:

* one for single machine processing using threads or processes
* and one for distributed memory clusters.

Documentation from [dask.pydata.org](http://dask.pydata.org):
* [Scheduler Overview](http://dask.pydata.org/en/latest/scheduler-overview.html)
* [Single-Machine Scheduler](http://dask.pydata.org/en/latest/shared.html)
* [Many-Machine Scheduler (aka "dask.distributed")](http://distributed.readthedocs.io/en/latest/)

## Profiling

Dask provides a few tools to help make debugging and profiling graph execution easier.

Documentation from [dask.pydata.org](http://dask.pydata.org):
* [Inspecting](http://dask.pydata.org/en/latest/inspect.html)
* [Diagnostics](http://dask.pydata.org/en/latest/diagnostics.html)

# Demonstration

In [1]:
import numpy as np

Recall the numpy interface, as it will be mimiced by Dask.

In [ ]:
np.

Prepare for demo by opening a system utility to monitor usage of CPU and memory.

In [ ]:
# Linux/Mac: from the terminal:
# ! top -o mem -O cpu

Now do some math and watch how system resources are used:

In [ ]:
# Recall the np.random module:
#     * array of random samples from a gaussian distribution
#     * median=`loc`, stddev=`scale`, number of samples = `size`

array = np.random.normal(loc=10, scale=1.0, size=1000 )
array[0:10]

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(array)

NameError: name 'array' is not defined

Let's use numpy as our baseline, to set expectations for the time and resources needed to draw a larger number of samples.

In [3]:
# %%timeit
# N = int(2e7) # <1 sec
N = int(2e8) # <10 sec
#N = int(2e9) # <100 sec

np.random.normal(10, 0.1, size=(N,) ).mean()

10.00000147880727

Now, do it with Dask:

In [4]:
import dask.array as da

Notice the interface and compare with numpy:

In [ ]:
da.

In [6]:
# %%timeit
#N = int(2e7) # 0.1 sec
#N = int(2e8) # 2 sec
N = int(2e9) # 15+ sec

chunk_size=int(1e6)
da.random.normal(10, 0.1, size=(N,), chunks=(chunk_size,) ).mean().compute()

10.000002169108269

# Further Reading

## Reference Links

Dask is part of the Blaze project supported and offered by
Continuum Analytics and others.

* [Blaze](http://continuum.io/open-source/blaze)
* [Continuum Analytics](http://continuum.io)
* [3-clause BSD license](https://github.com/dask/dask/blob/master/LICENSE.txt)
* [`#dask tag`](http://stackoverflow.com/questions/tagged/dask)
* [GitHub issue tracker](https://github.com/dask/dask/issues)
* [blaze-dev@continuum.io](https://groups.google.com/a/continuum.io/forum/#!forum/blaze-dev)
* [gitter chat room](https://gitter.im/dask/dask)
* [xarray](http://xray.readthedocs.org/en/stable/)
* [scikit-image](http://scikit-image.org/docs/stable/)
* [scikit-allel](https://scikits.appspot.com/scikit-allel)
* [pandas](http://pandas.pydata.org/pandas-docs/version/0.17.0/)
* [Distributed scheduler](http://distributed.readthedocs.org/en/latest/)

## Dask Tutorial

Links:
* [Matt Rocklin, core Dask developer, at PyData Seattle 2015 (YouTube)](https://www.youtube.com/watch?v=ieW3G7ZzRZ0)
* [Tutorial Repository (GitHub)](https://github.com/dask/dask-tutorial)

Set-Up:

```bash
git clone https://github.com/dask/dask-tutorial
cd dask-tutorial
python ./prep.py  # create artificial data sets

conda install dask pandas
pip install castra graphviz  # optional, for graphs
```

---
*Copyright Continuum 2012-2016 All Rights Reserved.*